In [1175]:
# archivo:    spsi.p6.rsa
# asignatura: Seguridad y Protección de Sistemas Informáticos
# práctica:   Práctica 6
# autor:      José María Martín Luque

In [1176]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

# Ejercicio 1

*Emita una pareja de claves, pública y privada, para adherirse usted a un círculo RSA como usuario $A$.*

Necesitamos:

- Dos números $p$, $q$ primos.
- El producto $n = pq$.
- El valor $\phi(n)$.
- Un valor $e$ tal que $(e, \phi(n)) = 1$.
- Un valor $d$ tal que $ed \equiv 1 \mod \phi(n)$, es decir, $d = e^{-1}\mod\phi(n)$.

La clave pública será la tupla $(n, e)$ y la clave privada, $(p, q, d)$.

In [1177]:
p_A = (2^19) - 1
print(p_A)
print(is_prime(p_A))

524287
True


In [1178]:
q_A = (2^31) - 1
print(q_A)
print(is_prime(q_A))

2147483647
True


In [1179]:
n_A = p_A*q_A; n_A

1125897758834689

Calculamos $\phi(n)$ sabiendo que $\phi(n) = (p-1)(q-1)$.

In [1180]:
phi_A = (p_A - 1)*(q_A - 1); phi_A

1125895610826756

Para buscar $e$ generamos enteros aleatorios menores que $\phi(n)$ con la función `random_element` hasta encontrar alguno tal que $(e, \phi(n)) = 1$.

In [1181]:
def genera_e(phi):
    e = ZZ.random_element(phi)
    while gcd(e, phi) != 1:
        e = ZZ.random_element(phi)
    return e

In [1182]:
e_A = genera_e(phi_A); e_A

60199173633307

In [1183]:
gcd(e_A, phi_A)

1

In [1184]:
def genera_d(e, phi):
    bezout = xgcd(e, phi); bezout
    d = Integer(mod(bezout[1], phi))
    return d

In [1185]:
d_A = genera_d(e_A, phi_A) ; d_A

735506640523015

In [1186]:
mod(d_A * e_A, phi_A)

1

La clave pública de $A$ va a ser la tupla $(n, e)$.

In [1187]:
pub_A = {"n": n_A, "e": e_A}; pub_A

{'e': 60199173633307, 'n': 1125897758834689}

La clave privada de $A$ va a ser la tupla $(p, q, d)$.

In [1188]:
priv_A = {"p": p_A, "q": q_A, "d": d_A}; priv_A

{'d': 735506640523015, 'p': 524287, 'q': 2147483647}

# Ejercicio 2

*Simule ser un segundo usuario $B$, del mismo círculo y envíe de $A$ a $B$ un mensaje firmado y cifrado. Al recibirlo descífrelo y compruebe la identidad del emisor.*

A continuación se describen algunas funciones auxiliares necesarias para codificar un mensaje.

In [1189]:
def aplana_texto(texto):
    texto = texto.lower()
    texto = texto.replace("ñ", "ny")
    texto = texto.replace("á", "a")
    texto = texto.replace("é", "e")
    texto = texto.replace("í", "i")
    texto = texto.replace("ó", "o")
    texto = texto.replace("ú", "u")
    texto = texto.replace(" ", "")
    return texto

In [1190]:
def texto_a_numeros(texto):
    """Convierte el texto recibido a una cadena de números.
    
    Cada número que representa la posición de cada carácter en el alfabeto —sin 
    la ñ ni acentos—, que es reemplazada por ny.

    El texto se codifica en minúscula.
    """

    texto = aplana_texto(texto)

    numeros = []
    for c in texto:
        n = ord(c) - 97
        numeros.append(n)
    
    return ''.join(format(x, '02') for x in numeros)

In [1191]:
def numeros_a_texto(numeros):
    """Convierte la cadena de números recibida en un string entendiendo que cada 
    número es la posición de una letra en el alfabeto.
    """
    texto = ""
    
    numeros_bloques = [numeros[i:i+2] for i in range(0, len(numeros), 2)]
    
    for bloque in numeros_bloques:
        c = chr(int(bloque) + 97)
        texto = texto + c
    return texto

In [1192]:
def bloques(lista, m):
    """Devuelve bloques de 'm' elementos a partir de 'lista'."""
    for i in range(0, len(lista), m):
        yield lista[i:i + m]

In [1193]:
def unir(lista_strings):
    """Dada una lista de strings, devuelve el string de la lista concatenada"""
    res = ""
    for string in lista_strings:
        res += string
    return res

Tenemos que generar las claves de $B$.

In [1194]:
p_B = (2^61) - 1
print(p_B)
print(is_prime(p_B))

2305843009213693951
True


In [1195]:
q_B = (2^89) - 1
print(q_B)
print(is_prime(q_B))

618970019642690137449562111
True


In [1196]:
n_B = p_B*q_B
phi_B = (p_B - 1)*(q_B - 1)
e_B = genera_e(phi_B)
d_B = genera_d(e_B, phi_B) ; d_B

1321078713497676344611631197413024695759808037

In [1197]:
pub_B = {"n": n_B, "e": e_B}; pub_B

{'e': 552184328085843760956484202252120028198071473,
 'n': 1427247692705959880439315947500961989719490561}

In [1198]:
priv_B = {"p": p_B, "q": q_B, "d": d_B}; priv_B

{'d': 1321078713497676344611631197413024695759808037,
 'p': 2305843009213693951,
 'q': 618970019642690137449562111}

In [1199]:
# Vamos a seguir el ejemplo de teoría

pub_B = {"n": 1735939, "e": 187};
priv_B = {"p": 1597, "q": 1087, "d": 37075}

Ahora vamos a cifrar el mensaje.

In [1200]:
mensaje2 = "repliegue al blocao a"

In [1201]:
def mensaje_num_a_bloques_cifras(mensaje_num, mod, cifrado=True):
    
    cifras_mod = len(str(mod))
    
    tam_mensaje = len(mensaje_num)
    if (cifrado):
        tam_bloque = cifras_mod - 1
    
        if (tam_bloque % 2 != 0):
            tam_bloque = tam_bloque - 1
    else:
        tam_bloque = cifras_mod
    
    if (tam_mensaje > tam_bloque):
        # Separa los números del mensaje en bloques de tamaño tam_mensaje
        bloques_mensaje = [mensaje_num[i:i+tam_bloque] for i in range(0, tam_mensaje, tam_bloque)]
        
        if (len(bloques_mensaje[-1]) != tam_bloque):
            bloques_mensaje[-1] = bloques_mensaje[-1].ljust(tam_bloque, '2')
        
        return bloques_mensaje
    else:
        if (len(mensaje_num) != tam_bloque):
            mensaje_num = mensaje_num.ljust(tam_bloque, '2')
        return [mensaje_num]

In [1202]:
def exp_cifrado_RSA(bloques, exp, mod):
    cifras_mod = len(str(mod))
    
    bloques_mensaje_cifrado = []
    
    for bloque in bloques:
        bloque_cifrado = str(power_mod(int(bloque), exp, mod))
        bloques_mensaje_cifrado.append(bloque_cifrado)
    
    for i, bloque in enumerate(bloques_mensaje_cifrado):
        bloques_mensaje_cifrado[i] = bloque.rjust(cifras_mod, '0')
    
    return bloques_mensaje_cifrado

In [1203]:
def exp_descifrado_RSA(bloques, exp, mod):
    tam_mayor = 0
    bloques_descifrados = []
    
    # Desciframos cada uno de los bloques
    # Almacenamos el tamaño del bloque mayor
    for bloque in bloques:
        bloque_descifrado = str(power_mod(int(bloque), exp, mod))
        
        if len(bloque_descifrado) > tam_mayor:
            tam_mayor = len(bloque_descifrado)
        
        bloques_descifrados.append(bloque_descifrado)
    
    # Completamos los bloques con 0 por la izquierda
    for i, bloque in enumerate(bloques_descifrados):
        bloques_descifrados[i] = bloque.rjust(tam_mayor, '0')
    
    return bloques_descifrados

In [1204]:
# Únicamente cifrado
mensaje2_bloques_cifras = mensaje_num_a_bloques_cifras(texto_a_numeros(mensaje2), pub_B["n"])
print(mensaje2_bloques_cifras)

mensaje2_cifrado = exp_cifrado_RSA(mensaje2_bloques_cifras, pub_B["e"], pub_B["n"])
print(mensaje2_cifrado)

mensaje2_descifrado_bloques_cifras = exp_descifrado_RSA(mensaje2_cifrado, priv_B["d"], pub_B["n"])
print(mensaje2_descifrado_bloques_cifras)

print(numeros_a_texto(unir(mensaje2_descifrado_bloques_cifras)))

['170415', '110804', '062004', '001101', '111402', '001400']
['1442861', '0082626', '0690931', '1489385', '0363461', '0062317']
['170415', '110804', '062004', '001101', '111402', '001400']
replieguealblocaoa


In [1205]:
# Cifrado y firma
mensaje22 = "repliegue al blocao a"
print("El mensaje original es: " + mensaje22 + "\n")
mensaje22_bloques_cifras = mensaje_num_a_bloques_cifras(texto_a_numeros(mensaje22), pub_A["n"])
print("mensaje original en bloques de cifras", mensaje22_bloques_cifras)

mensaje22_firmado = exp_cifrado_RSA(mensaje22_bloques_cifras, priv_A["d"], pub_A["n"])
print("mensaje firmado", mensaje22_firmado)

tmp = unir(mensaje22_firmado)
mensaje22_firmado_bloques_cifras = mensaje_num_a_bloques_cifras(tmp, pub_B["n"])
print("mensaje firmado en bloques de cifras", mensaje22_firmado_bloques_cifras)

mensaje22_firmado_cifrado = exp_cifrado_RSA(mensaje22_firmado_bloques_cifras, pub_B["e"], pub_B["n"])
print("mensaje firmado cifrado", mensaje22_firmado_cifrado)

mensaje22_firmado_descifrado = exp_descifrado_RSA(mensaje22_firmado_cifrado, priv_B["d"], pub_B["n"])
print("mensaje firmado descifrado", mensaje22_firmado_descifrado)

tmp2 = unir(mensaje22_firmado_descifrado)
mensaje22_firmado_descifrado_bloques_cifras = mensaje_num_a_bloques_cifras(tmp2, pub_A["n"], False)
print("mensaje firmado descifrado en bloques de cifras", mensaje22_firmado_descifrado_bloques_cifras)

mensaje22_verificado_descifrado = exp_descifrado_RSA(mensaje22_firmado_descifrado_bloques_cifras, pub_A["e"], pub_A["n"])
print("mensaje verificado descifrado", mensaje22_verificado_descifrado)

print("\nEl mensaje obtenido es: " + numeros_a_texto(unir(mensaje22_verificado_descifrado)))

El mensaje original es: repliegue al blocao a

('mensaje original en bloques de cifras', ['17041511080406', '20040011011114', '02001400222222'])
('mensaje firmado', ['0892491932540857', '0619379108068276', '0852714259697267'])
('mensaje firmado en bloques de cifras', ['089249', '193254', '085706', '193791', '080682', '760852', '714259', '697267'])
('mensaje firmado cifrado', ['1321948', '0779596', '0239577', '1455527', '1502909', '0678461', '0055751', '1266823'])
('mensaje firmado descifrado', ['089249', '193254', '085706', '193791', '080682', '760852', '714259', '697267'])
('mensaje firmado descifrado en bloques de cifras', ['0892491932540857', '0619379108068276', '0852714259697267'])
('mensaje verificado descifrado', ['17041511080406', '20040011011114', '02001400222222'])

El mensaje obtenido es: replieguealblocaoawww


# Ejercicio 3

*La clave pública de Alice en determinado círculo de usuarios RSA es $\langle 49569253, 3\rangle$. Cifre para ella un determinado mensaje $m$ de su invención.*

In [1206]:
pub_B = {"n": 49569253, "e": 3};

In [1207]:
mensaje3 = "La cerámica de Talavera no es cosa menor"
mensaje3_bloques_cifras = mensaje_num_a_bloques_cifras(texto_a_numeros(mensaje3), pub_B["n"])
print(mensaje3_bloques_cifras)

mensaje3_cifrado = exp_cifrado_RSA(mensaje3_bloques_cifras, pub_B["e"], pub_B["n"])
print(mensaje3_cifrado)

['110002', '041700', '120802', '000304', '190011', '002104', '170013', '140418', '021418', '001204', '131417']
['08761897', '25218492', '34060258', '28094464', '24288778', '44570553', '28801650', '43524731', '22578755', '10413809', '38223689']


# Ejercicio 4

*Por descuido, Alice y Bob se han unido a un círculo de usuarios RSA con las claves públicas:*
- *Alice: $\langle 49569253, 1121\rangle$*
- *Bob: $\langle 49569253, 1717\rangle$*

Eve decide enviar cierto mensaje $m$ a ambos usuarios, cifrándolo adecuadamente. A Alice le hace llegar $16187450$ y a Bob $2897699$. Carol es conocedora de esta comunicación y no se resiste a inmiscuirse en la conversación secreta, pudiendo desvelar lo oculto. Diga usted qué mensaje $m$ enviado por Eve descubrió Carol.

In [1208]:
# Fallo del protocolo por el módulo común
n = 49569253
eA = 1121
eB = 1717
mA = 16187450
mB = 2897699

# Hallamos los coeficientes de Bezout
d, u, v = xgcd(eA, eB)

mensaje = str(power_mod(mA, u, n) * power_mod(mB, v, n) % n)
print(mensaje)
print(numeros_a_texto(mensaje))

171800
rsa


# Ejercicio 5
*La clave pública de Alice, Bob y Eve en determinado círculo de usuarios RSA es, respectivamente:*
- *Alice: $\langle 49569253, 3\rangle$*
- *Bob: $\langle 45729283, 3\rangle$*
- *Eve: $\langle 62615533, 3\rangle$*

*Dave necesita enviar a los tres un mismo mensaje $m$, que ha cifrado resultando en cada caso:*
- *Alice: $14087331, 1176293, 46828916, 15244478, 34299711, 48939817$*
- *Bob: $27425305, 14483107, 13942406, 44901956, 24874564, 7906872$*
- *Eve: $62007067, 35550694, 62611116, 4185043, 52598842, 59059670$*

Carol está a la escucha y siente interés en conocer qué tiene que decir Dave a sus tres amigos. Recordando sus conocimientos adquiridos en SPSI de la carrera se pone manos a la obra y consigue leer el mensaje $m$. Diga usted qué mensaje $m$ descubrió Carol al inmiscuirse y poder leer el envío de Dave.

In [1209]:
# Fallor del protocolo por elección de exponentes bajos

e = 3
nA = 49569253
nB = 45729283
nE = 62615533
mA = [14087331, 1176293, 46828916, 15244478, 34299711, 48939817]
mB = [27425305, 14483107, 13942406, 44901956, 24874564, 7906872]
mE = [62007067, 35550694, 62611116, 4185043, 52598842, 59059670]

bloques_mensaje = []

# Resolvemos el sistema de cada trío de bloques y calculamos
# la raíz cúbica
for i in range(len(mA)):
    bloque = str((crt([mA[i], mB[i], mE[i]], [nA, nB, nE]).nth_root(e)))
    bloques_mensaje.append(bloque)

# Ajustamos con 0 para que todos los bloques tengan la misma
# longitud
tam_mayor = max([len(bloque) for bloque in bloques_mensaje])

for i, bloque in enumerate(bloques_mensaje):
        bloques_mensaje[i] = bloque.rjust(tam_mayor, '0')
    
print(bloques_mensaje)
print(numeros_a_texto(unir(bloques_mensaje)))

['1100131', '4020704', '0418030', '4111418', '0600191', '4182222']
lanocheesdelosgatosww


# Referencias

1. Minh Van Nguyen. *Number Theory and the RSA Public Key Cryptosystem*. Recuperado el 28 de noviembre de 2019 desde http://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html.